In [270]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
from models import *
from helpers import *
debug = False

In [283]:
# Our input $x$
TEXT = torchtext.data.Field()

# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

TEXT.build_vocab(train)
if debug:
    TEXT.build_vocab(train, max_size=1000)

train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [111]:
it = iter(test_iter)
batch = next(it)
print(batch.text.size())
# print(batch.text[:,3])
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,4].data]))
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,5].data]))

torch.Size([32, 10])
makes some executives nervous <eos> last year the research and development division of weyerhaeuser co. the large <unk> concern invited a <unk> to its <unk> wash. offices <eos> phil <unk> a software
more expensive than direct treasury borrowing said rep. <unk> stark d. calif. the bill 's chief sponsor <eos> the complex financing plan in the s&l bailout law includes raising $ N billion


Perplexity goals:
count: 120-200
feedforward: 100-150
recurrent: below 100 (between 80-100)

In [245]:
train_iter, _, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
tgram = Trigram(TEXT)
tgram.train_counts(train_iter)
tgram.set_alpha(0.25, 0.25)

Iteration 0


In [269]:
_, val_iter, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
for (a1, a2) in [(0.3, 0.5)]: #it.product(np.arange(0.1, 1, 0.1), repeat=2):
    if a1 + a2 >= 1:
        continue
    tgram.set_alpha(a1, a2)
    le = LangEvaluator(tgram, TEXT, evalmetric='perplexity')
    print(a1, a2, le.evaluate(val_iter))

Iteration 0
0.3 0.5 191.339042201


In [263]:
class NNLM(nn.Module):
    def __init__(self, TEXT, **kwargs):
        super(CNN, self).__init__()

        # Save parameters:
        self.activation = kwargs.get('activation', F.relu)
        
        # V is size of vocab, D is dim of embedding
        V = TEXT.vocab.vectors.size()[0]
        D = TEXT.vocab.vectors.size()[1]
        self.embeddings = nn.Embedding(V, D)
        self.embeddings.weight = nn.Parameter(
            TEXT.vocab.vectors, requires_grad= \
            kwargs.get('train_embeddings', True))
        
        in_channels = 1
        out_channels = 60
        kernel_sizes = [6] 

        # List of convolutional layers
        self.convs_inner = nn.ModuleList(
            [nn.Conv2d(in_channels, out_channels, (K, D),
                       padding=(K, 0)) for K in kernel_sizes])
        
        self.convs_direct = nn.ModuleList(
            [nn.Conv2d(in_channels, out_channels, (K, D),
                       padding=(K,0)) for K in kernel_sizes])

        self.dropout = nn.Dropout(kwargs.get('dropout', 0.5))
        
        self.linear = nn.Linear(len(kernel_sizes) * out_channels, V)
        
    def forward(x):
        x = self.embeddings(x) # [btch_sz, sent_len, D]
        x = x.unsqueeze(1) # [btch_sz, in_channels, sent_len, D]
        

8
[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]


In [276]:
def f(a, **kwargs):
    print(a)
    print(b)
    print(kwargs['b'])

In [278]:
f(10, **{'a' : 5})

TypeError: f() got multiple values for argument 'a'

In [127]:
a = torch.LongTensor([1,2,3])
c = tuple(a)
b = np.array([1,2,3])
d = tuple(b)
print(d == c)

True


## IGNORE STUFF BELOW HERE

In [128]:
    # NOT USED!
    # Here arr is a 1-D numpy array; this returns 
    # groups of n consecutive words (with overlapping)
    def get_ngrams(self, arr, n=3):
        len_ngrams = arr.shape[0] - n + 1
        ngram_inds = np.tile(np.reshape(np.arange(len_ngrams), [len_ngrams, 1]), [1, n]) + \
                    np.tile(np.reshape(np.arange(n), [1, n]), [len_ngrams, 1])
        return np.take(arr, ngram_inds)

In [145]:
np.exp(1)

2.7182818284590451

In [285]:
print(TEXT.vocab.vectors.size())

torch.Size([10001, 300])
